# Model Training (Machine Learning)

We train six traditional ML classifiers on nine feature-selection datasets:

1. **Recursive Feature Elimination**
2. **Select K Best**
3. **Fisher Score Chi-Square**
4. **Extra Trees Classifier**
5. **Pearson Correlation**
6. **Mutual Information**
7. **Mutual Info Regression**
8. **Manual Uniqueness**
9. **Variance Threshold**

Each dataset trains:
- Logistic Regression  
- Gradient Boosting Classifier  
- K-Nearest Neighbours  
- Random Forest Classifier  
- Decision Tree Classifier  
- Support Vector Machine  

Metrics → `Accuracy`, `Precision`, `Recall`, `F1`  
Visuals → Confusion Matrices

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

FEATURES_BASE = Path("../data/processed/features")
PROC_BASE = Path("../data/processed/ml")
MODEL_BASE = Path("../models/ml")
FIG_BASE = Path("../figures/ml")

for p in [PROC_BASE, MODEL_BASE, FIG_BASE]:
    p.mkdir(parents=True, exist_ok=True)

METHODS = ["rfe","skb","fscs","etc","pc","mi","mir","mu","vt"]
RANDOM_STATE = 42

## Helper Functions

In [2]:
def plot_and_save_confusion(y_true, y_pred, path, title):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(title)
    fig.tight_layout()
    fig.savefig(path, dpi=300, bbox_inches="tight")
    plt.close(fig)

def compute_metrics(y_true, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "F1": f1_score(y_true, y_pred, average="weighted", zero_division=0)
    }

## Model Definitions

In [3]:
MODELS = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Gradient Boosting": GradientBoostingClassifier(),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC()
}

## Train All Six Models Across Nine Feature Sets

In [4]:
for method in METHODS:
    print("\n" + "="*60)
    print(f"▶ Training ML models for: {method.upper()}")
    print("="*60)

    in_dir = FEATURES_BASE / method
    train_path = in_dir / "train.csv"
    test_path = in_dir / "test.csv"
    if not train_path.exists() or not test_path.exists():
        print(f"⚠️ Missing train/test for {method}, skipping.")
        continue

    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # clean / dtype
    train_df = train_df.dropna(subset=["DepressionEncoded"])
    test_df = test_df.dropna(subset=["DepressionEncoded"])
    X_train = train_df.drop(columns=["DepressionEncoded"])
    y_train = train_df["DepressionEncoded"].astype(int)
    X_test = test_df.drop(columns=["DepressionEncoded"])
    y_test = test_df["DepressionEncoded"].astype(int)

    results = []
    # create method-specific folders
    proc_out = PROC_BASE / method
    model_out = MODEL_BASE / method
    fig_out = FIG_BASE / method
    for p in [proc_out, model_out, fig_out]:
        p.mkdir(parents=True, exist_ok=True)

    for name, model in MODELS.items():
        print(f" - Training {name} ...")
        # fit
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics = compute_metrics(y_test, y_pred)
        metrics_row = {"Model": name, **metrics}
        results.append(metrics_row)

        # save confusion matrix
        cm_path = fig_out / f"{name.lower().replace(' ', '_')}_confusion.png"
        plot_and_save_confusion(y_test, y_pred, cm_path, f"{name} Confusion ({method.upper()})")

        # save model
        model_path = model_out / f"{name.lower().replace(' ', '_')}.pkl"
        joblib.dump(model, model_path)

    # save results CSV
    res_df = pd.DataFrame(results)
    res_df.to_csv(proc_out / "results_traditional_ml.csv", index=False)
    print(f"✅ Saved results for {method.upper()} to {proc_out / 'results_traditional_ml.csv'}")


▶ Training ML models for: RFE
 - Training Logistic Regression ...
 - Training Gradient Boosting ...
 - Training KNN ...
 - Training Random Forest ...
 - Training Decision Tree ...
 - Training SVM ...
✅ Saved results for RFE to ..\data\processed\ml\rfe\results_traditional_ml.csv

▶ Training ML models for: SKB
 - Training Logistic Regression ...
 - Training Gradient Boosting ...
 - Training KNN ...
 - Training Random Forest ...
 - Training Decision Tree ...
 - Training SVM ...
✅ Saved results for SKB to ..\data\processed\ml\skb\results_traditional_ml.csv

▶ Training ML models for: FSCS
 - Training Logistic Regression ...
 - Training Gradient Boosting ...
 - Training KNN ...
 - Training Random Forest ...
 - Training Decision Tree ...
 - Training SVM ...
✅ Saved results for FSCS to ..\data\processed\ml\fscs\results_traditional_ml.csv

▶ Training ML models for: ETC
 - Training Logistic Regression ...
 - Training Gradient Boosting ...
 - Training KNN ...
 - Training Random Forest ...
 - Tra

## Summary of Model Performance Across All Feature Sets

In [5]:
all_results = []

for method in METHODS:
    res_path = PROC_BASE / method / "results_traditional_ml.csv"
    if res_path.exists():
        df = pd.read_csv(res_path)
        df["Feature Set"] = method.upper()
        all_results.append(df)
    else:
        print(f"⚠️ Missing results for {method.upper()}")

if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    combined_results = combined_results.sort_values(["Feature Set", "Accuracy"], ascending=[True, False])
    
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    display(combined_results)

    # Save the full summary for documentation
    summary_out = PROC_BASE / "all_model_results_summary.csv"
    combined_results.to_csv(summary_out, index=False)
    print(f"✅ Combined model summary saved → {summary_out}")
else:
    print("⚠️ No model results found. Please run training first.")

,Model,Accuracy,Precision,Recall,F1,Feature Set
18,Logistic Regression,0.800000,0.798754,0.800000,0.798470,ETC
23,SVM,0.790123,0.790873,0.790123,0.789303,ETC
19,Gradient Boosting,0.765432,0.767866,0.765432,0.766305,ETC
21,Random Forest,0.750617,0.753958,0.750617,0.749150,ETC
20,KNN,0.698765,0.700839,0.698765,0.695469,ETC
22,Decision Tree,0.651852,0.662598,0.651852,0.653123,ETC
12,Logistic Regression,0.767901,0.771709,0.767901,0.768826,FSCS
13,Gradient Boosting,0.760494,0.766566,0.760494,0.761301,FSCS
17,SVM,0.760494,0.764876,0.760494,0.761422,FSCS
15,Random Forest,0.738272,0.746842,0.738272,0.738511,FSCS


✅ Combined model summary saved → ..\data\processed\ml\all_model_results_summary.csv
